# ObsAstro - Astrometry

Written by Macarena G. del Valle-Espinosa \
Last Update: January 12, 2024 by Macarena G. del Valle-Espinosa \
Observational Astronomy - Computer Lab - University of Edinburgh

## Introduction 

In this exercise, you will create the World Coordinate System of an image containing the trail of an asteroid. You will then be able to measure the celestial coordinates of the beginning and ending of the trail, and calculate the orbital radius using these coordinates. 

Here is the list of general packages we are going to use. \
Through the notebook you will see more specific packages and dedicated functions you will also need.

In [6]:
pip install matplotlib numpy scipy astropy

  Using cached astropy-7.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached pyerfa-2.0.1.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached astropy_iers_data-0.2025.2.10.0.33.26-py3-none-any.whl.metadata (5.1 kB)
Using cached astropy-7.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.3 MB)
Using cached astropy_iers_data-0.2025.2.10.0.33.26-py3-none-any.whl (1.9 MB)
Using cached pyerfa-2.0.1.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (738 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyyaml bokeh mpl_point_clicker

  Using cached mpl_point_clicker-0.4.1-py3-none-any.whl.metadata (1.2 kB)
Using cached mpl_point_clicker-0.4.1-py3-none-any.whl (6.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install lineid_plot pandas photutils

  Using cached lineid_plot-0.6-py2.py3-none-any.whl.metadata (704 bytes)
  Using cached photutils-2.1.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
Using cached lineid_plot-0.6-py2.py3-none-any.whl (9.6 kB)
Using cached photutils-2.1.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits

In [10]:
from bokeh.io import show, output_notebook
# set local host for bokeh (change the number to the one in the url at the top of the page)
localhost = 'localhost:8888'

In [11]:
output_notebook()

Loading BokehJS ...

<div class="alert alert-block alert-warning">

# <font color='red'>WARNING!</font> 
Through this notebook, you should replace any <font color='red'>'''text'''</font> with your own file/variable names. \
Check the `notebook_url` variable matches the one on your notebook (beginning of the url)


# Creating the WCS

The interfaces for this exercise are very similar to the photometry ones. \
Using the function `polyxenaPlot` you will be able to display the image with the polyxena asteroid. Using the 25 stars marked in the manual, you should get the pixel coordinates of their centres. As before, you will be able to zoom in and out, change the colormap values and get the coordinates with a left double-click

In [12]:
from astrometryExercise import polyxenaPlot

In [13]:
# Load your data here. 
# To load the data you can use the astropy.io function `fits.open()`

polyxena = fits.open("polyxena.fits")[0].data

In [14]:
polyxenaPlot(polyxena, notebook_url=localhost)

In [15]:
# Copy here the x and y coordinates you selected using the polyxenaPlot interface.
# WARNING! Make sure you have 25 pair of coordinates!

xCoor = np.array([1304.91, 1414.53, 2035.99, 2057.69, 1756.87, 1855.07, 2137.01, 1911.15, 1860.5, 1902.82, 1472.3, 1101.77, 1058.78, 2723.53, 2814.83, 2607.57, 1202.4, 1280.7, 1304.7, 1362.6, 1387.33, 2384.65, 2505.61, 2582.91, 2614.24])
yCoor = np.array([2398.7, 2291.13, 2143.49, 2139.42, 1923.5, 1873.00, 1917.80, 1737.34, 1424.24, 1327.31, 1510.78, 1753.02, 1814.51, 2130.29, 2054.09, 1299.26, 2105.21, 1973.55, 1900.38, 1852.26, 1755.7, 1549.53, 1682.19, 1585.95, 1667.48])

Let's check now how well the centres are located. The function `polyxenaPlotCentres` will show a grid with all the selected stars centred. The red cross marks the exact pixel coordinate selected. If one cross is not properly centred you should manually change the coordinates on the variables `xCoor` and `yCoor` defined above.

In [16]:
from astrometryExercise import polyxenaPlotCentres

In [17]:
polyxenaPlotCentres(polyxena, xCoor, yCoor, notebook_url=localhost)

Once you are happy with the location of your stars, it is time to create the World Coordinate System (WCS). The WCS will transform pixel values into RA and DEC coordinates. \
In order to create your WCS you need the pixel coordinates of your stars as well as the celestial coordinates (available in the manual).

In [18]:
# These are the imports needed to create the WCS
import astropy.units as u
import astropy.wcs as WCS
from astropy.coordinates import SkyCoord


In [19]:
# You should create now the arrays contaning the RA and DEC values of the stars

RA = np.array([
    '14:52:49.429', '14:52:40.927', '14:51:52.970', '14:51:51.318',
    '14:52:14.225', '14:52:06.541', '14:51:44.912', '14:52:02.075',
    '14:52:05.628', '14:52:02.219', '14:52:35.570', '14:53:04.314',
    '14:53:07.746', '14:51:00.138', '14:50:53.009', '14:51:07.926',
    '14:52:56.981', '14:52:50.848', '14:52:48.953', '14:52:44.432',
    '14:52:42.356', '14:51:25.420', '14:51:16.278', '14:51:10.221',
    '14:51:07.936'])

DEC = np.array([
    '-28:54:51.13', '-28:56:37.55', '-28:58:56.30', '-28:59:00.21',
    '-29:02:41.80', '-29:03:31.15', '-29:02:41.68', '-29:05:46.92',
    '-29:11:02.29', '-29:12:39.20', '-29:09:41.13', '-29:05:43.35',
    '-29:04:42.56', '-28:58:57.20', '-29:00:13.11', '-29:12:56.19',
    '-28:59:47.82', '-29:01:58.89', '-29:03:11.91', '-29:03:59.56',
    '-29:05:36.85', '-29:08:48.12', '-29:06:31.77', '-29:08:07.60',
    '-29:06:44.75'])


In [20]:
# Now we create the Sky Coordinates in a form the astropy.WCS will understand. 
# Give it a think on the units of your coordinates
#help(SkyCoord)

SkyCoords_wcs = SkyCoord(RA, DEC, unit=('hourangle', 'deg'))


# Finally, let's create the WCS with the stars pixels and celestial coordinates
polyxenaWCS = WCS.utils.fit_wcs_from_points((xCoor, yCoor), SkyCoords_wcs)

In [21]:
# As an example, you should check now the coordinates you recover when you use one of the known stars.
# The function 'wcs_pix2world' will help you with this process. 
# It needs the pixel coordinates of your object as input,
# and it will return the celestial coordinates based on your WCS

ind = 13

xcoorStar = xCoor[ind]
ycoorStar = yCoor[ind]

raStar, decStar = polyxenaWCS.wcs_pix2world(xcoorStar, ycoorStar, 0)

print(f'Original coordinates of the selected star: {SkyCoords_wcs.ra.value[ind]:.5f}, {SkyCoords_wcs.dec.value[ind]:.5f}')
print(f'Modeled  coordinates of the selected star: {raStar:.5f}, {decStar:.5f}')


Original coordinates of the selected star: 222.75057, -28.98256
Modeled  coordinates of the selected star: 222.75060, -28.98267


The celestial coordinates from `pix2world` should be close to the original ones, but not identical. \
We are going to quantified now how good (or bad) our WCS calibration is by calculating the RMS.


In [22]:
# You should create a function that calculates the RMS (in arcseconds) of the WCS.
# The function should be very similar to the standard deviation, but with 2 variables: RA and DEC
# Here are some hints:
# (1) Calculate first the difference (i.e. the distance) between the original points and the ones recovered 
#     from 'wcs_pix2world'. This will be your "individual error" of the model in each datapoint.
# (2) With the differences you can apply now the general formula for the standard deviation, i.e., 
#     add all differences in quadrature, divide by the total number of datapoints, and do the square root

# NOTE: we recommend the outputs of this function to be the overall RMS as well as the individual differences.

def calculateRMS():
    
    differences = []
    
    for i in range(len(xCoor)):
        xcoorStar = xCoor[i]
        ycoorStar = yCoor[i]

        model_raStar, model_decStar = polyxenaWCS.wcs_pix2world(xcoorStar, ycoorStar, 0)

        model_raStar *= (u.deg).to(u.arcsec)
        model_decStar *= (u.deg).to(u.arcsec)
        
        og_raStar = (SkyCoords_wcs.ra).to(u.arcsec).value[i]
        og_decStar = (SkyCoords_wcs.dec).to(u.arcsec).value[i]

        #print(og_raStar)
        #print(model_raStar)
        # Change all to radians
        # model_raStar = np.radians(model_raStar)
        # model_decStar = np.radians(model_decStar)
        # og_raStar = np.radians(og_raStar)
        # og_decStar = np.radians(og_decStar)

        # According to lab manual = equation 3 is in degrees, calculation is in degrees, and factor of 15 is unneccesary!!!
    
        #print(og_raStar)

        raDiff = og_raStar - model_raStar # in degrees
        decDiff = og_decStar - model_decStar # in degrees
        #print(decDiff)
        #print(raDiff)
        #print(decDiff)

        angSep = np.sqrt(decDiff**2 + (np.cos(np.deg2rad(model_decStar / 3600))*raDiff)**2) # in degrees

        #print(np.deg2rad(model_decStar / 3600))
        #angSep = np.deg2rad(angSep) # convert to radians

        #print(angSep)

        differences.append(angSep)
        
    differences = np.array(differences)

    #print(np.sum(differences**2))
    #print(len(differences))
    rms = np.sqrt(np.sum(differences**2) / len(differences))
    #print(rms)

    #ms = np.rad2deg(rms)
    #print(rms)
    #rms = rms * 3600

    return rms, differences

In [23]:
# Using your previous function, calculate the RMS and the individual differences

rms, differences = calculateRMS()

print(differences)

print(f'The overall RMS is {rms}')

[0.16841774 0.34508817 0.22256131 0.14304731 0.74463932 0.32648014
 0.37358447 0.14967583 0.42154715 0.19893689 0.47478477 0.61073988
 0.56149877 0.43234369 0.57132644 0.69997563 0.25879522 0.16402461
 0.60924792 0.37538228 0.66137433 0.35456192 0.46249136 0.1406304
 0.45170917]
The overall RMS is 0.43733719092544604


### <font color='blue'>If your RMS is smaller than 0.5arcseconds...</font> congrats! You can move now to the next part of the exercise. You can skip the few following blocks and skip to "Getting the asteroid coordinates".
### <font color='red'>If your RMS is bigger than 0.5arcseconds...</font> we need to improve our WCS calibration. Use the following blocks to do so.

Using the "individual errors" from your function above we are going to check which stars are degrading our model. We will remove them from our original list, create a new model and check the RMS again. \
<font color='red'> <b>WARNING!</b></font>  Only do this if you are 100% happy with the stars centres

In [24]:
# Let's have a look first to the array
print(differences)

[0.16841774 0.34508817 0.22256131 0.14304731 0.74463932 0.32648014
 0.37358447 0.14967583 0.42154715 0.19893689 0.47478477 0.61073988
 0.56149877 0.43234369 0.57132644 0.69997563 0.25879522 0.16402461
 0.60924792 0.37538228 0.66137433 0.35456192 0.46249136 0.1406304
 0.45170917]


In [25]:
# You should have most of your points (~15) with a difference below 0.6. 
# If this is not the case you will need to improve the centres location. Talk with one of the TAs to help you with this
# Otherwise, let's create a mask to only keep the stars with differences below 0.6

rmsMask = [15]

# And now we repeat the creation of the WCS but using only the good stars
polyxenaWCS = WCS.utils.fit_wcs_from_points((xCoor[rmsMask], yCoor[rmsMask]), SkyCoords_wcs[rmsMask])

ValueError: `x0` is infeasible.

In [ ]:
# Finally, let's check if our model has improved. 
# You can use the same function you defined above, but remember to use only the good set of stars

rms, rmsArray = calculateRMS('''your inputs masked''')

print(f'The new RMS is {rms:.3f}')

__________

# Getting the asteroid coordinates

Now that we have the WCS we are able to transform the pixel coordinates of the asteroid into celestial. \
Using the function `polyxenaPlot` you can display again the image of the asteroid (which should be located in the centre of the field of view) and double-click on the beginning and ending of the trail to get the coordinates. \
Once you have the pixel coordinates, you can use again the `wcs_pix2world` to convert the pixel values into celestial coordinates.

In [28]:
polyxenaPlot(polyxena, notebook_url=localhost)

In [29]:
# Introduce here your pixel coordinates for the asteroid trail

polyxenaXLeft = 1780.43
polyxenaYLeft = 1828.69

polyxenaXRight = 1828.57
polyxenaYRight = 1831.91


In [30]:
# Using the wcs_pix2world function, calculate the celestial coordinates

polyxenaRa_left, polyxenaDec_left = polyxenaWCS.wcs_pix2world(polyxenaXLeft, polyxenaYLeft, 0)
polyxenaRa_right, polyxenaDec_right = polyxenaWCS.wcs_pix2world(polyxenaXRight, polyxenaYRight, 0)


In [31]:
print(f'Coordinates on left-side:  {polyxenaRa_left}, {polyxenaDec_left}')
print(f'Coordinates on right-side: {polyxenaRa_right}, {polyxenaDec_right}')

Coordinates on left-side:  223.05103655237866, -29.07140899368988
Coordinates on right-side: 223.03562760229516, -29.070297041010246


### And from here you should follow the instructions on the manual

In [32]:
decDiff = polyxenaDec_right - polyxenaDec_left
raDiff = polyxenaRa_right - polyxenaRa_left


angSep = np.sqrt(decDiff**2 + (np.cos(np.deg2rad(polyxenaDec_right))*raDiff)**2) # in degrees
angSep = np.radians(angSep) # in radians
print(angSep)
t = 90 * 60 # in seconds
v_E = (2*np.pi)/(31558140) # in rad per sec
print(v_E)
v_A = angSep / t
e_v_A = np.radians(rms/3600) / t
a_min = ((- 1 - np.sqrt(1 + ((4 * v_E * t) / angSep ))) / 2)**2 
e_a_min = np.abs(((v_E * t)/angSep**2) * (1 + (1 + ((4 * v_E * t)/angSep))**(-1/2))) * np.radians(rms/3600)
a_plus = ((- 1 + np.sqrt(1 + ((4 * v_E * t) / angSep ))) / 2)**2

v_A_2 = v_E / np.sqrt(a_plus)

0.000235857185252673
1.990987208745378e-07


In [33]:
print(f'a_minus = {a_min}')
print(f'e_a_min = {e_a_min}')
print(f'a_plus = {a_plus}')

print(f'v_A = {v_A}')
print(f'e_v_A = {e_v_A}')
print((f'v_A_2 = {v_A_2}'))

a_minus = 7.251215269478493
e_a_min = 0.050322251571577735
a_plus = 2.865599141502607
v_A = 4.367725652827278e-08
e_v_A = 3.9264269151605455e-10
v_A_2 = 1.1761446859220295e-07
